In [15]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression;  log=LogisticRegression();
from xgboost import XGBClassifier;  xgb=XGBClassifier(n_estimators=10);
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import confusion_matrix, classification_report, roc_curve,roc_auc_score, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

train = pd.read_csv('joineddata.csv')
train.head()

,id,loanAmnt,term,interestRate,installment,grade,subGrade,employmentTitle,employmentLength,homeOwnership,...,n9,n10,n11,n12,n13,n14,endDate,avg_GDP,avg_UNRATE,avg_CPIAUCSL
0,0,35000.0,5,19.52,917.97,E,E2,320.0,2 years,2,...,2.0,7.0,0.0,0.0,0.0,2.0,2019-07-01,19510.161857,4.638095,244.072190
1,1,18000.0,5,18.49,461.90,D,D2,219843.0,5 years,0,...,NaN,13.0,NaN,NaN,NaN,NaN,2017-08-01,18605.830600,5.845000,237.282150
2,2,12000.0,5,16.99,298.17,D,D3,31698.0,8 years,0,...,3.0,11.0,0.0,0.0,0.0,4.0,2020-10-01,19923.215667,5.152381,249.167714
3,3,11000.0,3,7.26,340.96,A,A4,46854.0,10+ years,1,...,6.0,9.0,0.0,0.0,0.0,1.0,2018-08-01,19531.501000,4.500000,243.766583
4,4,3000.0,3,12.99,101.07,C,C2,54.0,NaN,1,...,7.0,12.0,0.0,0.0,0.0,4.0,2019-03-01,19767.011667,4.333333,246.275333


In [16]:
train.isnull().sum()[train.isnull().sum()>0].sort_values(ascending=False)

n11                   69752
employmentLength      46799
n8                    40271
n14                   40270
n13                   40270
n12                   40270
n9                    40270
n0                    40270
n1                    40270
n2                    40270
n3                    40270
n5                    40270
n6                    40270
n7                    40270
n10                   33239
n4                    33239
revolUtil               531
pubRecBankruptcies      405
dti                     239
title                     1
postCode                  1
employmentTitle           1
dtype: int64

In [17]:
#dti cannot be negative
train.loc[train["dti"]<0,"dti"]=0
#extract the year of employmwntlenght
train["employmentLength"]=train["employmentLength"].str.replace("< 1 year","0")
train["employmentLength"]=train["employmentLength"].str.replace("years","")
train["employmentLength"]=train["employmentLength"].str.replace("year","")
train["employmentLength"]=train["employmentLength"].str.replace("+","")
train["employmentLength"]=train["employmentLength"].str.replace(" ","")
train["employmentLength"]=train["employmentLength"].astype("float64")
# change the regioncode into object
train["regionCode"]=train["regionCode"].astype("object")
#trans the issue date in to year
train["issueDate"]=train["issueDate"].str.replace("-",".",1)
train["issueDate"]=train["issueDate"].str.replace("-01","")
train["issueDate"]=train["issueDate"].str.replace("20","")
train["issueDate"]=train["issueDate"].astype("float64")

In [18]:
#cover earliesCreditLine_month into year month
month=('Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec')
number=('01',"02",'03','04','05','06','07','08','09','10','11','12')
temp_month=dict(zip(month,number))

train[["earliesCreditLine_month","earliesCreditLine_year"]]=train["earliesCreditLine"].str.split("-",expand=True)
train["earliesCreditLine_month"]=train["earliesCreditLine_month"].map(temp_month)
train["earliesCreditLine"]=train["earliesCreditLine_year"]+"."+train["earliesCreditLine_month"]
train=train.drop(["earliesCreditLine_month","earliesCreditLine_year"],axis=1)
train["earliesCreditLine"]=train["earliesCreditLine"].astype("float64")
train["earliesCreditLine"]=train["earliesCreditLine"]-1944

In [19]:
# based on the grade and subgrade, ordinal encoding is used
subgrade_mapping = {f"{grade}{subgrade}": 7 - (i + subgrade / 10)
                    for i, grade in enumerate("ABCDEFG")
                    for subgrade in range(1, 6)}
train['subGrade'] = train['subGrade'].replace(subgrade_mapping)
train = train.drop('grade', axis=1)

In [20]:
train.head()

,id,loanAmnt,term,interestRate,installment,subGrade,employmentTitle,employmentLength,homeOwnership,annualIncome,...,n9,n10,n11,n12,n13,n14,endDate,avg_GDP,avg_UNRATE,avg_CPIAUCSL
0,0,35000.0,5,19.52,917.97,2.8,320.0,2.0,2,110000.0,...,2.0,7.0,0.0,0.0,0.0,2.0,2019-07-01,19510.161857,4.638095,244.072190
1,1,18000.0,5,18.49,461.90,3.8,219843.0,5.0,0,46000.0,...,NaN,13.0,NaN,NaN,NaN,NaN,2017-08-01,18605.830600,5.845000,237.282150
2,2,12000.0,5,16.99,298.17,3.7,31698.0,8.0,0,74000.0,...,3.0,11.0,0.0,0.0,0.0,4.0,2020-10-01,19923.215667,5.152381,249.167714
3,3,11000.0,3,7.26,340.96,6.6,46854.0,10.0,1,118000.0,...,6.0,9.0,0.0,0.0,0.0,1.0,2018-08-01,19531.501000,4.500000,243.766583
4,4,3000.0,3,12.99,101.07,4.8,54.0,NaN,1,29000.0,...,7.0,12.0,0.0,0.0,0.0,4.0,2019-03-01,19767.011667,4.333333,246.275333


In [21]:
train.loc[:,"n0":"n14"]=train.loc[:,"n0":"n14"].fillna(0)

In [26]:
train = train.drop('endDate', axis=1)
train.head()

,id,loanAmnt,term,interestRate,installment,subGrade,employmentTitle,employmentLength,homeOwnership,annualIncome,...,n8,n9,n10,n11,n12,n13,n14,avg_GDP,avg_UNRATE,avg_CPIAUCSL
0,0,35000.0,5,19.52,917.97,2.8,320.0,2.0,2,110000.0,...,12.0,2.0,7.0,0.0,0.0,0.0,2.0,19510.161857,4.638095,244.072190
1,1,18000.0,5,18.49,461.90,3.8,219843.0,5.0,0,46000.0,...,0.0,0.0,13.0,0.0,0.0,0.0,0.0,18605.830600,5.845000,237.282150
2,2,12000.0,5,16.99,298.17,3.7,31698.0,8.0,0,74000.0,...,5.0,3.0,11.0,0.0,0.0,0.0,4.0,19923.215667,5.152381,249.167714
3,3,11000.0,3,7.26,340.96,6.6,46854.0,10.0,1,118000.0,...,21.0,6.0,9.0,0.0,0.0,0.0,1.0,19531.501000,4.500000,243.766583
4,4,3000.0,3,12.99,101.07,4.8,54.0,NaN,1,29000.0,...,15.0,7.0,12.0,0.0,0.0,0.0,4.0,19767.011667,4.333333,246.275333


In [27]:
train.loc[:,"n0":"n14"]=train.loc[:,"n0":"n14"].fillna(0)
train.corr()["employmentLength"].sort_values()

earliesCreditLine    -0.214540
homeOwnership        -0.128706
applicationType      -0.040109
purpose              -0.037831
n6                   -0.034145
avg_UNRATE           -0.033481
isDefault            -0.015822
initialListStatus    -0.015678
employmentTitle      -0.013586
title                -0.004879
interestRate         -0.003302
n13                   0.000055
id                    0.000474
regionCode            0.003762
n11                   0.003862
subGrade              0.004276
n12                   0.007545
issueDate             0.012494
n0                    0.017362
ficoRangeLow          0.018688
ficoRangeHigh         0.018689
delinquency_2years    0.021592
avg_CPIAUCSL          0.021666
avg_GDP               0.025221
dti                   0.025623
pubRecBankruptcies    0.027587
verificationStatus    0.029196
pubRec                0.031045
n14                   0.035583
revolUtil             0.035735
postCode              0.039103
openAcc               0.042909
n10     

In [28]:
train.corr(numeric_only=False)["isDefault"].sort_values()

subGrade             -0.266415
ficoRangeLow         -0.130994
ficoRangeHigh        -0.130993
annualIncome         -0.042782
avg_UNRATE           -0.035857
employmentTitle      -0.030714
purpose              -0.029208
title                -0.025173
revolBal             -0.020929
employmentLength     -0.015822
totalAcc             -0.012756
initialListStatus    -0.007383
n5                   -0.006860
id                   -0.000351
n11                  -0.000295
regionCode            0.002822
n12                   0.003572
n8                    0.005403
postCode              0.007140
n13                   0.011021
n6                    0.012333
n0                    0.013164
applicationType       0.018197
delinquency_2years    0.020185
n4                    0.021352
pubRecBankruptcies    0.026038
pubRec                0.026558
openAcc               0.026867
n10                   0.034654
n7                    0.039918
earliesCreditLine     0.043096
n1                    0.047285
installm

In [29]:
#select the features with more than 0.1 corelation to predict the employmentLength
employmentLengthPre=train[["earliesCreditLine","homeOwnership","totalAcc","employmentLength","n7","n5","n2","n3","n9","n8"]]
train_employmentLengthPre=employmentLengthPre[employmentLengthPre["employmentLength"].notnull()]
test_employmentLengthPre=employmentLengthPre[employmentLengthPre["employmentLength"].isnull()]
x_train_employmentLengthPre=train_employmentLengthPre.drop(["employmentLength"],axis=1)
y_train_employmentLengthPre=train_employmentLengthPre["employmentLength"]
x_test_employmentLengthPre=test_employmentLengthPre.drop(["employmentLength"],axis=1)

#applied the random forest to predict the model
from sklearn.ensemble import RandomForestRegressor
ran1=RandomForestRegressor()
ran1.fit(x_train_employmentLengthPre,y_train_employmentLengthPre)
pre_employmentLengthPre=ran1.predict(x_test_employmentLengthPre)

train.loc[train["employmentLength"].isnull(),"employmentLength"]=pre_employmentLengthPre

In [30]:
# missing less than 1000
train[["dti","pubRecBankruptcies","revolUtil"]]=train[["dti","pubRecBankruptcies","revolUtil"]].fillna(0)

In [31]:
#release the ram
ran1=[]

In [32]:
#normalize the feature with extreme large or small value
from sklearn.preprocessing import StandardScaler
train_preprocessing=train
train_preprocessing[["loanAmnt","installment","employmentTitle","annualIncome","postCode","dti","ficoRangeLow","ficoRangeHigh","revolBal","revolUtil","totalAcc","title"]]= StandardScaler().fit_transform(train_preprocessing[["loanAmnt","installment","employmentTitle","annualIncome","postCode","dti","ficoRangeLow","ficoRangeHigh","revolBal","revolUtil","totalAcc","title"]])
train_preprocessing = train_preprocessing.astype('float32')

In [33]:
train_preprocessing_feature=train_preprocessing.drop(["id"],axis=1)

In [34]:
train_preprocessing_feature

,loanAmnt,term,interestRate,installment,subGrade,employmentTitle,employmentLength,homeOwnership,annualIncome,verificationStatus,...,n8,n9,n10,n11,n12,n13,n14,avg_GDP,avg_UNRATE,avg_CPIAUCSL
0,2.361518,5.0,19.52,1.835928,2.8,-0.672561,2.00,2.0,0.491187,2.0,...,12.0,2.0,7.0,0.0,0.0,0.0,2.0,19510.162109,4.638095,244.072189
1,0.411100,5.0,18.49,0.091610,3.8,1.387032,5.00,0.0,-0.437056,2.0,...,0.0,0.0,13.0,0.0,0.0,0.0,0.0,18605.830078,5.845000,237.282150
2,-0.277283,5.0,16.99,-0.534604,3.7,-0.378169,8.00,0.0,-0.030950,2.0,...,5.0,3.0,11.0,0.0,0.0,0.0,4.0,19923.214844,5.152381,249.167709
3,-0.392013,3.0,7.26,-0.370946,6.6,-0.235973,10.00,1.0,0.607217,1.0,...,21.0,6.0,9.0,0.0,0.0,0.0,1.0,19531.501953,4.500000,243.766586
4,-1.309857,3.0,12.99,-1.288447,4.8,-0.675057,8.26,1.0,-0.683621,2.0,...,15.0,7.0,12.0,0.0,0.0,0.0,4.0,19767.011719,4.333333,246.275330
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799995,1.214213,3.0,14.49,1.615780,4.6,-0.650617,7.00,1.0,-0.059957,0.0,...,13.0,10.0,14.0,0.0,0.0,0.0,3.0,19964.886719,4.176923,248.246765
799996,0.296370,3.0,7.90,0.359490,6.6,-0.401559,10.00,0.0,0.331645,2.0,...,19.0,2.0,7.0,0.0,0.0,0.0,0.0,18453.539062,6.000000,236.033081
799997,-0.965666,3.0,13.33,-0.898139,4.7,-0.651339,10.00,1.0,-0.161484,2.0,...,10.0,4.0,5.0,0.0,0.0,1.0,4.0,19590.992188,4.446154,244.459305
799998,0.548777,3.0,6.92,0.589735,6.6,-0.674147,10.00,0.0,0.288134,2.0,...,22.0,8.0,16.0,0.0,0.0,0.0,5.0,19299.253906,4.733333,241.501160


In [35]:
train_preprocessing_feature.corr()["isDefault"].sort_values()

subGrade             -0.266415
ficoRangeLow         -0.130994
ficoRangeHigh        -0.130993
annualIncome         -0.042782
avg_UNRATE           -0.035857
employmentTitle      -0.030714
purpose              -0.029208
title                -0.025173
revolBal             -0.020929
employmentLength     -0.015391
totalAcc             -0.012756
initialListStatus    -0.007383
n5                   -0.006860
n11                  -0.000295
regionCode            0.002822
n12                   0.003572
n8                    0.005403
postCode              0.007140
n13                   0.011021
n6                    0.012333
n0                    0.013164
applicationType       0.018197
delinquency_2years    0.020185
n4                    0.021352
pubRecBankruptcies    0.026042
pubRec                0.026558
openAcc               0.026867
n10                   0.034654
n7                    0.039918
earliesCreditLine     0.043096
n1                    0.047285
installment           0.051524
issueDat

In [36]:
train_preprocessing_feature=train_preprocessing_feature.drop(["initialListStatus","n5","n11","n12","n8","postCode","policyCode"],axis=1)

In [37]:
train_preprocessing_feature = train_preprocessing_feature.dropna(subset=['employmentTitle','title'])

In [38]:
train_preprocessing_feature=pd.get_dummies(train_preprocessing_feature)

In [39]:
x_train_full_preprocessing_feature=train_preprocessing_feature.drop(["isDefault"],axis=1)
y_train_full_preprocessing_feature=train_preprocessing_feature["isDefault"]

In [40]:
x_train_full_preprocessing_feature.isnull().sum()[x_train_full_preprocessing_feature.isnull().sum()>0].sort_values(ascending=False)

Series([], dtype: int64)

In [41]:
# Export the df to a csv

train.to_csv('Final_version_cleaned.csv', index=False)